In [ ]:
import pandas as pd

df = pd.read_table("/content/drive/MyDrive/shopping.txt",encoding='UTF-8',header=None)

df.columns = ['ratings','describe']
print(len(df['describe']),df['describe'].nunique())

df.drop_duplicates(subset=['describe'],inplace=True)
print(len(df['describe'])) #19908개로 중복제거

200000 199908
199908


In [ ]:
df.head(5)

,ratings,describe
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [ ]:
def func(x) :
  if x>2 :
    return 1
  return 0

# df['ratings'].unique() #1,2,4,5 총 4개의 unique값이 행에 존재하는 것을 알 수 있다.
df['binary'] = df['ratings'].apply(lambda x: func(x)) #binary -> 0과 1로 값을 나타내는 행
df

,ratings,describe,binary
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1
...,...,...,...
199995,2,장마라그런가!!! 달지않아요,0
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...,1
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요,1
199998,5,넘이쁘고 쎄련되보이네요~,1


In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 30.5 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
okt = Okt()
# print(okt.morphs('안녕하세요 송준호입니다.'))

In [ ]:
df['describe'] = df['describe'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣\s]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

stopword_okt_replace = []

for sentence in df['describe'] :
  morphed_sentence = okt.morphs(sentence)
  stopword_okt_replace.append([word for word in morphed_sentence if word not in stopwords])

<ipython-input-6-8cb3241a2247>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['describe'] = df['describe'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣\s]","")


In [ ]:
print(stopword_okt_replace[0])

['배공', '빠르고', '굿']


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
x_data = df['describe']
y_data = df['binary']

x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.2,stratify=y_data,random_state=902)

In [ ]:
print(len(x_train),len(x_test))

159926 39982


In [ ]:
token = Tokenizer()
token.fit_on_texts(x_train)
x_train_enc = token.texts_to_sequences(x_train)

word_to_index = token.word_index

In [ ]:
max_len = max(len(s) for s in x_train_enc)
vocabulary_size = len(word_to_index)+1
x_train_padded = pad_sequences(x_train_enc,maxlen=max_len)

x_train_padded[0]
y_train = to_categorical(y_train,vocabulary_size)

In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

model.add(Embedding(vocabulary_size,100))
model.add(LSTM(256))
model.add(Dense(vocabulary_size, activation = 'softmax'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

history = model.fit(x_train_padded,y_train,epochs=10,batch_size=256,validation_split=0.2)

